实现批量输入和目标的数据集类

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # 编码整个文本
        # 循环遍历数据，使用滑动窗口将数据分成重叠的最大长度序列
        # 假设token_ids的长度大于max_length
        for i in range(0,len(token_ids) - max_length, stride):
            # 输入的区域是i到i+maxlength，左闭右开，因此最后一个token不参与输入
            input_chunk = token_ids[i:i + max_length]
            # 第一个token不作为预测目标
            target_chunk = token_ids[i+1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    # 返回数据集的行数
    def __len__(self):
        return len(self.input_ids)
    # 返回数据集中单独一行数据
    def __getitem__(self, idx):
        return self.input_ids[idx],self.target_ids[idx]

数据加载器，用于生成具有输入对的批次

In [6]:
import tiktoken
def create_dataloader_v1(txt, 
                         batch_size=4, 
                         max_length=256, 
                         stride=8, 
                         shuffle=True, 
                         drop_last=True, 
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2") # 初始化tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) # 创建数据集
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, # drop_last=True时，如果最后一个批次小于指定的 batch_size，则丢弃它，以防止训练期间出现损失峰值。
        num_workers=num_workers # 用于预处理的 CPU 进程数
    )

    return dataloader

测试Dataset和DataLoader

In [7]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader) # 将数据加载器转换为 Python 迭代器，以便通过 Python 内置的 next() 函数获取下一个条目
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


理解 stride=1 的含义

In [8]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


练习 2.2 具有不同步长和上下文大小的数据加载器
为了更直观地了解数据加载器的工作原理，请尝试使用不同的设置运行它，例如 max_length=2 和 stride=2，以及 max_length=8 和 stride=2。

In [10]:
dataloader2_2 = create_dataloader_v1(raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)

data_iter2 = iter(dataloader2_2) 
first_batch2 = next(data_iter2)
print(first_batch2)
second_batch2 = next(data_iter2)
print(second_batch2)

[tensor([[ 40, 367]]), tensor([[ 367, 2885]])]
[tensor([[2885, 1464]]), tensor([[1464, 1807]])]


In [11]:
dataloader8_2 = create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)

data_iter3 = iter(dataloader8_2) 
first_batch3 = next(data_iter3)
print(first_batch3)
second_batch3 = next(data_iter3)
print(second_batch3)

[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]
[tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]


In [12]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader) # 将数据加载器转换为 Python 迭代器，以便通过 Python 内置的 next() 函数获取下一个条目
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
